In [1]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
# jupyter notebook warning 무시하기

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False

from glob import glob
import re
import math

In [225]:
file_list = []
for i in range(2015,2017):
    a = '../../data/team/해양환경측정망_'+str(i)+'.xlsx'
    file_list.append(a)
file_list

['../../data/team/해양환경측정망_2015.xlsx', '../../data/team/해양환경측정망_2016.xlsx']

In [226]:
df = pd.read_excel('../../data/team/해양환경측정망_2016.xlsx',skiprows = 3, header=None)
ex = df[df[2].str.contains('\n',na=False)]


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
347,2016.0,2.0,울산\n특별\n관리\n(남해),울산연안,1,NaN,11.4805,10.5581,34.1237,34.1224,...,18.817,29.68405,26.782760,364.784,353.640,7.35,22.15,0.322,0.380,3.5
368,2016.0,2.0,부산\n특별\n관리\n(남해),부산연안,1,NaN,11.4833,11.5000,34.0070,34.0599,...,17.608,23.75344,26.898080,347.984,346.136,12.20,8.95,0.836,0.742,3.5
391,2016.0,2.0,마산만\n특별\n관리\n(남해),행 암 만,1,NaN,7.5602,7.6105,32.8257,32.8723,...,2.201,25.66676,12.916667,21.000,19.348,9.70,13.60,3.380,5.172,4.0
412,2016.0,2.0,광양만\n특별\n관리\n(남해),광 양 만,1,NaN,6.3577,6.3861,31.5896,31.6105,...,3.069,12.30297,16.947080,114.184,122.976,11.10,13.90,9.864,9.948,2.0
424,2016.0,2.0,가막만\n환경\n보전\n(남해),가 막 만,1,NaN,6.2743,6.0253,32.6347,32.6697,...,9.207,25.18781,16.965060,189.364,185.836,7.35,11.20,1.400,1.460,2.8
429,2016.0,2.0,득량만\n환경\n보전\n(남해),득 량 만,1,NaN,5.5496,5.3778,31.5470,31.7299,...,1.457,36.16305,28.046320,5.208,16.100,13.10,22.50,5.320,6.800,1.2
434,2016.0,2.0,완도\n도암만\n환경\n보전\n(남해),완도연안,1,NaN,8.0433,8.5145,32.9106,33.1125,...,15.097,14.88279,16.502850,335.020,376.488,12.90,14.60,0.696,0.712,1.0
445,2016.0,2.0,함평만\n환경\n(서해),함평연안,1,NaN,3.8485,3.8341,31.6093,31.6024,...,17.856,49.58047,38.209670,593.292,666.960,23.30,25.10,0.676,0.904,0.2
449,2016.0,2.0,시화호\n인천\n특별 관리\n(서해),인천연안,1,NaN,2.7745,2.8800,30.2244,30.2596,...,37.696,110.47594,253.381600,716.856,708.260,9.60,29.20,1.310,1.120,0.4
482,2016.0,2.0,항만\n측정망\n(동해),거진연안,H1,NaN,4.2662,4.2814,33.7869,33.7884,...,7.874,17.99023,14.511410,172.200,199.864,5.35,3.90,0.596,0.502,5.0


In [227]:
# 전처리 1
key = ['2월','5월','8월','11월']
con_data = pd.DataFrame()
for i in file_list : 
    df_all = pd.read_excel(i, skiprows = 3, header=None, sheet_name=None)
    for j in key:
        # 월별시트 호출
        data = df_all[j]
        
        # h제거
        h= data[data[4].str.contains('H', na=False)]
        h[4] = h[4].str.get(i=1)
        h1 = h.index
        for h_1 in range(len(h1)):
            data.loc[h1[h_1],4] = h.loc[h1[h_1],4]
            
        # 항만제거
        man = data[data[2].str.contains('\n', na=False)]
        man[2] = man[2].str[-3:-1]
        h2 = man.index
        for h_2 in h2:
            data.loc[h_2,2] = man.loc[h_2,2]
            
        
        # 1로 시작하는 지역의 인덱스 추출
        index = data[data[4] == 1].index
        
        # index, st를 통해 빈칸채우기
        for z in range(len(index)):  
            insert_l = data.loc[index[z],:3]
            
            # 마지막인덱스 이외의 인덱스
            if index[z] < index[len(index)-1]:
                
                # 항만 중 이중으로 열병합 되어있는 경우 0:3까지 nan처리됨
                # nan값은 nan == nan 일시 Flase가 나옴
                # 0:3까지의 값은 이전 인덱스의 값과 동일하기 때문에 이전 인덱스의 값 호출
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for k in range(index[z],index[z+1]):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[k,:3] = insert_l_1
                else :
                    # 1이 연속으로 붙어있는데 한 카테고리일 경우
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for a_1 in range(index[z],len(data)):
                            data.loc[a_1,:2] = insert_l_1
                    else:
                        
                        for a in range(index[z],index[z+1]):
                            data.loc[a,:3] = insert_l
                        
            # 마지막 인덱스
            else : 
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for p in range(index[z],len(data)):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[p,:3] = insert_l_1
                else:
                    
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for e_1 in range(index[z],len(data)):
                            data.loc[e_1,:2] = insert_l_1
                    else: 
                        
                        for e in range(index[z],len(data)):
                            data.loc[e,:3] = insert_l
                    
                    
        con_data = pd.concat([con_data, data])
        # 빈칸채우기 끝
    

In [228]:
con_data.to_csv('../../data/team/1516.csv', index=False, encoding = 'utf-8')
con_data.to_csv('../../data/team/1516_r.csv', index=False, encoding = 'ANSI')

In [229]:
df = pd.read_csv('../../data/team/1516.csv', encoding='utf-8')

index_df = pd.read_excel('../../data/team/해양환경측정망_2019.xlsx')
index_list = index_df.loc[0,:]

for i in range(37):
    df.rename(columns = {str(i): index_list[i]}, inplace = True)
df

,년,월,생태구역,연안명칭,St.\nNo\n,해수수질기준,표층수온,저층수온,표층염분,저층염분,...,저층DIP,표층T-P,저층T-P,표층SiO2-Si,저층SiO2-Si,표층SS,저층SS,표층Chl-a,저층Chl-a,표층투명도
0,2015.0,2.0,동 해,거진연안,1,NaN,5.3747,4.3124,34.0458,33.9969,...,9.672,21.04218,19.81086,225.000,207.540,1.0,1.3,0.564,0.564,15.0
1,2015.0,2.0,동 해,거진연안,2,NaN,5.1215,4.9201,33.9709,34.0076,...,9.889,20.68537,21.83516,222.700,223.100,1.9,1.8,0.486,0.362,13.0
2,2015.0,2.0,동 해,속초연안,1,NaN,5.1585,5.3037,33.9601,34.0315,...,11.656,25.20982,23.42577,249.500,247.020,1.5,1.6,0.286,0.410,13.0
3,2015.0,2.0,동 해,속초연안,2,NaN,5.4283,5.3188,34.0491,34.0469,...,12.028,25.58926,22.79399,250.900,251.220,2.3,1.9,0.390,0.452,13.0
4,2015.0,2.0,동 해,속초연안,3,NaN,5.4563,5.3802,34.0606,34.0528,...,12.245,22.01837,22.34480,259.100,250.880,2.1,1.4,0.414,0.474,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4731,2016.0,11.0,남해,낙동강하구,6,NaN,15.9414,17.0331,25.0252,32.0719,...,35.712,42.40800,68.88200,509.740,667.996,18.2,24.4,0.948,0.724,1.3
4732,2016.0,11.0,남해,낙동강하구,7,NaN,16.0315,16.4359,27.0753,31.4207,...,22.227,43.55500,36.33200,492.016,514.864,9.8,16.4,0.788,0.292,1.4
4733,2016.0,11.0,남해,낙동강하구,8,NaN,15.6902,16.6588,24.0356,31.9845,...,20.863,37.32400,35.24700,505.372,495.320,9.6,13.0,0.888,0.414,1.3
4734,2016.0,11.0,남해,낙동강하구,9,NaN,17.0742,17.0219,32.2541,32.2689,...,19.840,27.68300,29.20200,475.720,475.944,8.7,10.5,0.470,0.384,1.5


In [230]:
b = df
for i in range(len(b)):
    if b.loc[i,'생태구역'] =='동 해':
        b.loc[i,'생태구역'] = '동해'
    if b.loc[i,'생태구역'] =='서 해':
        b.loc[i,'생태구역'] = '서해'
    if b.loc[i,'생태구역'] =='남 해':
        b.loc[i,'생태구역'] = '남해'
        

In [231]:
b['해수수질기준'] = 0

In [232]:
b = b.dropna()
b

,년,월,생태구역,연안명칭,St.\nNo\n,해수수질기준,표층수온,저층수온,표층염분,저층염분,...,저층DIP,표층T-P,저층T-P,표층SiO2-Si,저층SiO2-Si,표층SS,저층SS,표층Chl-a,저층Chl-a,표층투명도
0,2015.0,2.0,동해,거진연안,1,0,5.3747,4.3124,34.0458,33.9969,...,9.672,21.04218,19.81086,225.000,207.540,1.0,1.3,0.564,0.564,15.0
1,2015.0,2.0,동해,거진연안,2,0,5.1215,4.9201,33.9709,34.0076,...,9.889,20.68537,21.83516,222.700,223.100,1.9,1.8,0.486,0.362,13.0
2,2015.0,2.0,동해,속초연안,1,0,5.1585,5.3037,33.9601,34.0315,...,11.656,25.20982,23.42577,249.500,247.020,1.5,1.6,0.286,0.410,13.0
3,2015.0,2.0,동해,속초연안,2,0,5.4283,5.3188,34.0491,34.0469,...,12.028,25.58926,22.79399,250.900,251.220,2.3,1.9,0.390,0.452,13.0
4,2015.0,2.0,동해,속초연안,3,0,5.4563,5.3802,34.0606,34.0528,...,12.245,22.01837,22.34480,259.100,250.880,2.1,1.4,0.414,0.474,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4731,2016.0,11.0,남해,낙동강하구,6,0,15.9414,17.0331,25.0252,32.0719,...,35.712,42.40800,68.88200,509.740,667.996,18.2,24.4,0.948,0.724,1.3
4732,2016.0,11.0,남해,낙동강하구,7,0,16.0315,16.4359,27.0753,31.4207,...,22.227,43.55500,36.33200,492.016,514.864,9.8,16.4,0.788,0.292,1.4
4733,2016.0,11.0,남해,낙동강하구,8,0,15.6902,16.6588,24.0356,31.9845,...,20.863,37.32400,35.24700,505.372,495.320,9.6,13.0,0.888,0.414,1.3
4734,2016.0,11.0,남해,낙동강하구,9,0,17.0742,17.0219,32.2541,32.2689,...,19.840,27.68300,29.20200,475.720,475.944,8.7,10.5,0.470,0.384,1.5


In [234]:
b.to_csv('../../data/team/1516_1.csv', index=False, encoding = 'utf-8')
b.to_csv('../../data/team/1516_1_R.csv', index=False, encoding = 'ANSI')

In [260]:
a = pd.read_csv('../../data/team/1516_1.csv', encoding='utf-8', engine='python')

In [261]:
c = pd.read_csv('../../data/team/해양환경측정망_maxmin.csv', encoding = 'utf-8')

In [262]:
list_a = c['연안명칭'].unique()[:65]

In [263]:
for i in range(len(a)):
    for j in list_a:
        if a.loc[i,'연안명칭'] == j:
            a.loc[i,'생태구역'] = c[c['연안명칭'] == j]['생태구역'].iloc[0]

In [265]:
a['Ds'] = ''
for i in range(len(a)):
    a.loc[i,'Ds'] = aaa(a.loc[i,'저층수온'],a.loc[i,'저층염분'])
    

In [268]:
a['Ds']

0       10.3664
1       10.2136
2        10.118
3       10.1133
4        10.098
         ...   
4141    7.94517
4142    8.07072
4143    8.00793
4144    7.93747
4145    7.94962
Name: Ds, Length: 4146, dtype: object

In [267]:
a['저층DO']

0       10.497195
1       10.592401
2       10.644849
3       10.403763
4       10.332221
          ...    
4141     6.287336
4142     7.578891
4143     7.464735
4144     7.378833
4145     7.563831
Name: 저층DO, Length: 4146, dtype: float64

In [ ]:
do = a['저층DO']/a['Ds']

din = a['표층DIN']
dip = a['표층DIP']
chl = a['표층Chl-a']
w = a['표층투명도']

In [287]:
chl_a = []
din_a = []
dip_a = []
w_a = []
do_a = []
for i in range(len(a)):
    chl_a.append(chl_convert(i))
    din_a.append(din_convert(i))
    dip_a.append(dip_convert(i))
    w_a.append(w_convert(i))
    do_a.append(do_convert(i))
su = []

In [288]:
for i in range(len(a)):
    su.append(cul(do_a[i],chl_a[i],w_a[i],din_a[i],dip_a[i]))

In [309]:
for i in range(len(a)):
    a.loc[i,'해수수질기준'] = classfy(su[i])
    

In [ ]:
loc_list = a['연안명칭'].unique()
list_name = ['연안','만','하구','만','.자란만','동안','남안','외안','호']
a['구군'] = a['연안명칭']
for j in list_name:
    name_length = a['연안명칭'].str.find(j)

    for i in name_length.index :
        if name_length[i] != -1:
            a.loc[i,'구군'] = a.loc[i, '연안명칭'][0:name_length[i]]
        

In [317]:
a['해수수질기준'].value_counts()

1    1621
2    1265
3     823
4     284
5     153
Name: 해수수질기준, dtype: int64

In [314]:
a.to_csv('../../data/team/해양환경측정망_15,16.csv',index=False, encoding='utf-8')
a.to_csv('../../data/team/해양환경측정망_15,16_R.csv',index=False, encoding='ANSI')

In [308]:
def cul(a,b,c,d,e):
    sum = 10*a+6*(b+c)/2+4*(d+e)/2
    return sum

def classfy(i):
    if i <=23:
        return 1
    elif (i >23)&(i<=33):
        return 2
    elif (i >33)&(i<=46):
        return 3
    elif (i >46)&(i<=59):
        return 4
    elif (i >59):
        return 5
    
def aaa(t, S) : 
    T = 273.15 + t
    
    C = -173.4292 + 249.6339*(10**2 * T**-1) + 143.3483 * math.log(T * 10**-2) -21.8492*(T * 10**-2) -S*(0.033096 - 0.014259* T *10**-2 + 0.0017 * T**2 * 10**-4 )
    c = math.e**C
    result = c*32/22.4
    return result
    

def chl_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 2.1
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 6.3
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 3.7
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 1.6
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 2.2
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5

# DO 계수 
def do_convert(a1):
        b = 0.9
        if do[a1] >= b :
            return 1
        elif (do[a1] < b)&(do[a1] >= b - b*0.1):
            return 2
        elif (do[a1] < b - b*0.1)&(do[a1] >= b - b*0.25) :
            return 3
        elif (do[a1] < b - b*0.25)&(do[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    
        
        
        
def dip_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 20
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 35
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 25
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 15
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 30
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
def din_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 140
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 220
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 230
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 165
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 425
        if chl[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
def w_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 8.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 2.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 0.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 8.0
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 1.0
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5